<style>
body {
    max-width: 900px;
    margin: 40px auto;
    padding: 0 20px;
    font-family: "Georgia", serif;
    line-height: 1.6;
}
</style>

<div style="text-align: center; padding: 60px 60px">
  <h1 style="font-weight: bold; font-size: 3.1em">
    GA-UFLP-1
  </h1>
</div>

In [ ]:
from cargar_datos import *
from ga2 import *
from graficas import *

In [ ]:
random_seed = [0, 1, 2, 3, 4, 5, 6, 7, 8]
fitness_objetivo = 257896

path = "resultados"

In [2]:
import random

In [3]:
configuraciones = [
    # Mejor resultado
    {
        "tamaño_de_poblacion": 300,
        "tasa_de_mutacion": 1,
        "tasa_de_crossover": 0.8,
        "tamaño_del_torneo": 40,
        "maximo_de_generaciones": 1000,
        "tipo_de_crossover": "un punto",
        "tipo_de_mutacion": "mejor"
    },
    # Pues
    {   
        # asignar random de 100 a 200
        "tamaño_de_poblacion": random.randint(100, 200),
        "tasa_de_mutacion": random.uniform(0, 1),
        "tasa_de_crossover": random.uniform(0, 1),
        "tamaño_del_torneo": random.randint(10, 50),
        "maximo_de_generaciones": 1000,
        "tipo_de_crossover": "uniforme",
        "tipo_de_mutacion": "mejor"
    },
    # Solucion mas rapida
    {
        "tamaño_de_poblacion": 30,
        "tasa_de_mutacion": 1,
        "tasa_de_crossover": .9,
        "tamaño_del_torneo": 50,
        "maximo_de_generaciones": 1000,
        "tipo_de_crossover": "uniforme",
        "tipo_de_mutacion": "random"
    }
]

In [ ]:
for i in range(1, 10):
    file_name = f'UFLP-{i}'
    

    n_instalaciones, n_clientes, costos_fijos, costos_de_transporte = cargar_datos('datos/'+file_name+'.txt')

    resultados = {}
    os.makedirs(os.path.join(path, file_name), exist_ok=True)
   

    # Ejecucion del algoritmo genetico
    print(f"Ejecutando GA para {file_name} con semilla aleatoria: {random_seed}\n")

    for config in configuraciones:
        nombre_configuracion = f"N={config['tamaño_de_poblacion']}, M={config['tasa_de_mutacion']}, C={config['tasa_de_crossover']}, T={config['tamaño_del_torneo']}, G={config['maximo_de_generaciones']}, tc={config['tipo_de_crossover']}"
        print(f"Ejecutando configuracion: {nombre_configuracion}")
        resultados[nombre_configuracion] = []

        for ejecucion in range(num_ejecuciones):
            ga = UFLP_GA(n_instalaciones, n_clientes, costos_fijos, costos_de_transporte,
                        config["tamaño_de_poblacion"], config["tasa_de_mutacion"], config["tasa_de_crossover"],
                        config["tamaño_del_torneo"], config["maximo_de_generaciones"])
            if config["tipo_de_crossover"] == "uniforme":
                ga._operador_de_crossover = ga._crossover_uniforme
            elif config["tipo_de_crossover"] == "un punto":
                ga._operador_de_crossover = ga._crossover_un_punto
            else:
                raise ValueError(f"Tipo de crossover desconocido: {config['tipo_de_crossover']}")

            mejor_solucion, mejor_fitness, historial_de_fitnesses, tiempo_de_ejecucion, num_generaciones = ga.run()
            resultados[nombre_configuracion].append({
                "mejor_solucion": mejor_solucion+1,
                "mejor_costo": mejor_fitness,
                "tiempo_de_ejecucion": tiempo_de_ejecucion,
                "num_generaciones": num_generaciones,
                "historial_de_fitnesses": historial_de_fitnesses
            })
            print(f"{ejecucion + 1}/{num_ejecuciones}  ")
    print("\n")


    # Resumen
    tablas_resumen = {}
    for nombre_configuracion, resultados_ejecuciones in resultados.items():
        costos = [res["mejor_costo"] for res in resultados_ejecuciones]
        tiempos = [res["tiempo_de_ejecucion"] for res in resultados_ejecuciones]
        mejor_costo = min(costos)
        peor_costo = max(costos)
        costo_promedio = np.mean(costos)
        desviacion_estandar_costo = np.std(costos)
        tiempo_promedio = np.mean(tiempos)
        desviacion_porcentual = (peor_costo - mejor_costo) / mejor_costo * 100 if mejor_costo != 0 else np.inf

        tablas_resumen[nombre_configuracion] = pd.DataFrame({
            "Métrica": ["Mejor Costo", "Peor Costo", "Costo Promedio", "Desviacion Estandar Costo", "Tiempo Promedio de Ejecucion (s)", "Desviacion Porcentual (%)"],
            "Valor": [f"{mejor_costo:.2f}", f"{peor_costo:.2f}", f"{costo_promedio:.2f}", f"{desviacion_estandar_costo:.2f}", f"{tiempo_promedio:.2f}", f"{desviacion_porcentual:.2f}"]
        })

        os.makedirs(os.path.join(path, file_name, nombre_configuracion), exist_ok=True)

    for nombre_configuracion, resumen_df in tablas_resumen.items():
        resumen_df.to_csv(os.path.join(path, file_name, nombre_configuracion, f"resumen.csv"), index=False)


    # Graficar resultados
    curva_corvengencia_individual(resultados, path, file_name)
    box_plot_costos(resultados, path, file_name)
    guardar_resultados(resultados, path, file_name, random_seed)

    curva_convergencia_comparacion(resultados, path, file_name)
    box_plot_costos_comparacion(resultados, path, file_name)
    resumen_tablas(resultados, path, file_name, num_ejecuciones, tablas_resumen, random_seed)


Ejecutando GA para UFLP-2 con semilla aleatoria: 8

Ejecutando configuracion: N=300, M=1, C=0, T=30, G=1000, tc=un punto
1/9  
2/9  
3/9  
4/9  
5/9  
6/9  
7/9  
8/9  
9/9  
Ejecutando configuracion: N=150, M=0.75, C=0.25, T=10, G=1000, tc=uniforme
1/9  
2/9  
3/9  
4/9  
5/9  
6/9  
7/9  
8/9  
9/9  
Ejecutando configuracion: N=30, M=1, C=0, T=10, G=1000, tc=un punto
1/9  
2/9  
3/9  
4/9  
5/9  
6/9  
7/9  
8/9  
9/9  


Texto guardado en resultados\UFLP-2\N=300, M=1, C=0, T=30, G=1000, tc=un punto\mejores_soluciones.txt
Texto guardado en resultados\UFLP-2\N=150, M=0.75, C=0.25, T=10, G=1000, tc=uniforme\mejores_soluciones.txt
Texto guardado en resultados\UFLP-2\N=30, M=1, C=0, T=10, G=1000, tc=un punto\mejores_soluciones.txt
Ejecutando GA para UFLP-3 con semilla aleatoria: 8

Ejecutando configuracion: N=300, M=1, C=0, T=30, G=1000, tc=un punto
1/9  
2/9  
3/9  
4/9  
5/9  
6/9  
7/9  
8/9  
9/9  
Ejecutando configuracion: N=150, M=0.75, C=0.25, T=10, G=1000, tc=uniforme
1/9  
2/9  